In [197]:
txt_file = raw_input('Enter the name of your output file: ')

try:
    my_output_file = open(txt_file, "w")
except IOError as e:
    print("I/O error({0}): {1}".format(e.errno, e.strerror))

if not my_output_file.closed:
#     my_output_file.write("#1\n")
#     my_output_file.write("double({},{})\n".format(len(text_list), 2))
    for line in text_list:
        my_output_file.write(str(line).replace('\n', '') + '\n')
    print('File Successfully written.')
    my_output_file.close()

Enter the name of your output file: scraped data/content_3.txt
File Successfully written.


In [199]:
from gensim.models import doc2vec
import nltk
import re

def split_sentence(sentence):
    words = re.split('\W+', sentence.lower())
    return [word for word in words if word != ""]

class MyDocs(object):
    def __iter__(self):
        for i, text in enumerate(open("scraped data/content_3.txt")):
            yield doc2vec.LabeledSentence(words=split_sentence(text), tags=['%s' % i])

# Train the doc2vec model
mydocs = MyDocs()
model = doc2vec.Doc2Vec(mydocs, size = 200, window = 8, min_count = 5, workers = 4)
model.save('review.model')

In [278]:
model = doc2vec.Doc2Vec.load('review.model')
print model.most_similar(positive=["pita", "bagel"], negative=["spicy"], topn=3)

[('juice', 0.6861406564712524), ('sandwich', 0.6588626503944397), ('oatmeal', 0.6480457782745361)]


In [201]:
import numpy as np
# Calculate the cosine similarity between two vecotrs 
def cossim(v1, v2):
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2))

# Get the specific line of the review text file
def get_line(filename, n):
    with open(filename) as f:
        for i, e in enumerate(f):
            if i == n:
                return e
# Get the index of top n most similar review
def argmaxn(l, n):
    l_copy = list(l)
    args = []
    for i in range(n):
        arg = np.argmax(l_copy)
        args.append(arg)
        l_copy[arg] = -float('inf')
    return args

In [280]:
# Calculate the vector of input text according to our training model
input_text = "great vege burger and good juice"
input_vec = model.infer_vector(split_sentence(input_text))

# Compute the cosine similarity values between the input text and all archived reviews
cossims_with_input = map(lambda v: cossim(input_vec, v), model.docvecs)

# Print the reviews with the highest cosine similarity values
sim_ids = argmaxn(cossims_with_input, 3)
for i in range(3):
    print "(Review-%s, Similarity:%.4f):" % (sim_ids[i] + 1, cossims_with_input[sim_ids[i]]),\
          get_line('content_3.txt', sim_ids[i])

(Review-549, Similarity:0.7823): "546","There's loads of good places to eat on 46th St, but we chose Bare Burger based on the good reviews. Plenty of choice of burgers, (you can pretty much create your own), good beer on tap and friendly service from our server Sean. That and the fact it's well priced make it a good family place to eat in mid-town."

(Review-5557, Similarity:0.7692): "5554","We wanted to try a cuban place and it lived up to the reviews, Nice people, full, buzzy, decent food, great mohitos. and not expensive. We booked which was smart as all other tables taken. Cuba Libre ...."

(Review-11477, Similarity:0.7686): More "



In [220]:
import numpy as np
import pandas as pd

In [230]:
np.sum([1,2,3])

6

In [235]:
len(input_vec)

200

In [282]:
# Construct a data frame with the restaurants and document-vectors of all archived reviews
restaurant_textvecs = pd.read_csv('restaurant.csv')
restaurant_textvecs['Textvec'] = model.docvecs
import numpy as np
# Compute similarity scores between the input text and the restaurants (defined here as the cosine similarity between
# the document-vector of the input text and the sum/mean of the document-vectors of all the reviews of a restaurant --
# no difference between sum and mean)
#restaurant_cossims = restaurant_textvecs.groupby('restaurant_name').agg(lambda v: cossim(np.sum(v), input_vec))
restaurant_cossims = restaurant_textvecs.groupby('restaurant_name')['Textvec'].agg(lambda v: cossim(np.sum(v),input_vec))
# Print the reviews with the highest similarity scores
sim_ids = argmaxn(restaurant_cossims, 3)
#for i in range(3):
for i in range(len(sim_ids)):
    restaurant_name = restaurant_cossims.index[sim_ids[i]]
    print "Restaurant is: %s" % restaurant_name, "and the similarity is equal to: %.4f" % restaurant_cossims[restaurant_name]
    

Restaurant is: Bar B and the similarity is equal to: 0.6811
Restaurant is: Jivamuktea Cafe and the similarity is equal to: 0.6761
Restaurant is: Spreads and the similarity is equal to: 0.6724


In [255]:
restaurant_textvecs.groupby('restaurant_name')['Textvec'].agg(lambda v: cossim(np.sum(v),input_vec))
# input_vec
# restaurant_textvecs

restaurant_name
12 Chairs                         0.832781
456 Shanghai Cuisine              0.812132
55 Fulton Market                  0.754476
5th & Mad                         0.728163
A Asosan Sushi                    0.778533
ABC Home Grown                    0.570072
Absolute Bagels                   0.776071
Adyar Ananda Bhavan               0.790933
Agra Indian & Pakistan Rest       0.800352
Al Dente Restaurant               0.766462
Alidoro                           0.742553
Alin Sushi                        0.727775
Amba                              0.824673
Amish Market                      0.739935
Amsterdam Gourmet                 0.768167
Amura                             0.820937
An Choi                           0.812487
Anatolia Mediterranean Cuisine    0.787820
Angelica Kitchen                  0.821578
Angkor Cambodian Bistro           0.781041
Ardesia Wine Bar                  0.775339
Asia De Cuba                      0.771514
Aux Epices                        0.75

In [ ]:
pd.read_csv('restaurant.csv', index_col=0).shape

In [ ]:
df = pd.read_csv('scraped data/content_3.csv', index_col=0)

In [204]:
!wc -l "scraped data/content_3.txt"

   27607 scraped data/content_3.txt


In [206]:
restaurant_textvecs['restaurant_ID']